# Notebook Settings

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

# Imports

``` ipython
import sys
sys.path.insert(0, '../../')

import torch
import gc
import pandas as pd
from time import perf_counter

from src.network import Network
from src.decode import decode_bump
from src.utils import clear_cache
```

# Helpers

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

``` ipython
import numpy as np
def get_theta(a, b, GM=0, IF_NORM=0):

    u, v = a, b

    if GM:
        v = b - np.dot(b, a) / np.dot(a, a) * a

    if IF_NORM:
        u = a / np.linalg.norm(a)
        v = b / np.linalg.norm(b)

    return np.arctan2(v, u)
```

``` ipython
def get_idx(model):
    ksi = model.PHI0.cpu().detach().numpy()
    print(ksi.shape)

    theta = get_theta(ksi[0], ksi[2], GM=0, IF_NORM=0)
    return theta.argsort()
```

``` ipython
def get_overlap(model, rates):
    ksi = model.PHI0.cpu().detach().numpy()
    return rates @ ksi.T / rates.shape[-1]

```

# Parameters

``` ipython
REPO_ROOT = '/home/leon/models/NeuroTorch/'
conf_name = 'config_EI.yml'
```

# Exploring Parameter Space

To find parameters for which we have a multistable ring attractor, we
use torch **batching** capabilities to run parallel simulations across
the parameter space. The idea is that we will create "batches" of
parameters and pass them to the model.

## Batching a single parameter

``` ipython
model = Network(conf_name, REPO_ROOT, IF_STP=1, DT=0.001, VERBOSE=0, LIVE_FF_UPDATE=1, I0=[1, 0])
```

With torch we can easily pass lists of parameters or batches to the
model. Here, let's batch the recurrent strenght $J_{EE}$.

``` ipython
N_BATCH = 20
# Here we pass a list of parameters to J_STP which is JEE for the model with stp
model.J_STP = torch.linspace(0, 10, N_BATCH, dtype=torch.float32, device='cuda')

# For consistency we need to add a dummy extra dimension
# This is so that the models performs dot products correctly
# In the model J_STP is multiplied by rates of size (N_BATCH * N_NEURON)
# (N_BATCH * 1) * (N_BATCH * N_NEURON) = (N_BATCH * N_NEURON)

model.J_STP = model.J_STP.unsqueeze(-1)
# we need to scale J_STP correctly 1/sqrt(K)
model.J_STP = model.J_STP * model.Jab[0, 0]  
print('Jee', model.J_STP.shape)

# We set the number of batches
model.N_BATCH = N_BATCH
# and run the model

start = perf_counter()
rates_Jee = model().cpu().detach().numpy()
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
print('rates', rates_Jee.shape)
```

``` ipython
idx = get_idx(model)
rates_ordered = rates_Jee[..., idx]

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(model.J_STP.cpu(), m0[:, -1], '-o')
ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$<Rates>_i$')

ax[1].plot(rates_Jee.mean(-1).T)
ax[1].set_xlabel('$J_{EE}$')
ax[1].set_ylabel('Rates')
plt.show()
```

``` ipython
print(model.J_STP.shape, m1.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(model.J_STP.cpu(), m1[:, -1])
ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$\mathcal{F}_1$')

ax[1].plot(m1.T)
ax[1].set_xlabel('$Step$')
ax[1].set_ylabel('$\mathcal{F}_1$')
plt.show()
```

Here, for example, with J<sub>STP</sub>=10 we have a ring attractor!

``` ipython

```

## Batching multiple parameters

### Simuls

Sometimes we won't be so lucky and need to search harder over multiple
parameters. In order to **batch** over multiple parameters, we need to
carefully create each parameter batch. Here, let's batch the recurrent
strenght $J_{EE}$ and the feedforward strength $J_{E0}$.

``` ipython
model = Network(conf_name, REPO_ROOT, IF_STP=1, VERBOSE=0, LIVE_FF_UPDATE=1, N_BATCH=1, I0=[.2, 0])
```

First we create the lists of parameters to sweep

``` ipython
N_JEE = 20
N_JE0 = 20

JEE_list = np.linspace(0, 20, N_JEE).astype(np.float32)
print('Jee list', JEE_list)

JE0_list = np.linspace(1, 3, N_JE0).astype(np.float32)
print('Je0 list', JE0_list)

JEE = torch.from_numpy(JEE_list).to('cuda')
JE0 = torch.from_numpy(JE0_list).to('cuda')
```

Now we need to expand these lists into tensors with the correct shapes.
To do so we create a two new tensors J<sub>EE</sub> and J<sub>E0</sub>
of size (N<sub>JEE</sub>, N<sub>JE0</sub>) where each row of
J<sub>EE</sub> is a repetition of Jee list and each column of Je0 is a
copy of Je0 list. In that way, all the values of J<sub>EE</sub> are
associated once with a value of Je0.

``` ipython
JEE = JEE.unsqueeze(0).expand(N_JE0, N_JEE)
print('JEE first col', JEE[0])

JE0 = JE0.unsqueeze(1).expand(N_JE0, N_JEE)
print('JE0 first row', JE0[:, 0])
```

Torch models need a single batch dimension so we concatenate the two
dimensions into tensors of size
(N<sub>BATCH</sub>=N<sub>JEE</sub>\*N<sub>JE0</sub>, 1) We need the
extra dummy dimension so that in the model dot products are done
properly.

``` ipython
JEE = JEE.reshape((-1, 1)) 
print('JEE', JEE.shape)

JE0 = JE0.reshape((-1, 1)) 
print('JE0', JE0.shape)
```

Now we need to set the number of batches and copy our tensors to the
model

``` ipython
N_BATCH = N_JE0 * N_JEE
# Here we need to do some work on Ja0 first,
# since it has two dimensions for E and I and we need to repeat the I values
Ja0 = model.Ja0.repeat((N_BATCH, 1, 1))
print('Ja0', Ja0.shape)

# now we can pass JE0 to Ja0
# we need to scale JaE properly
Ja0[:,0] = JE0 * model.M0 * torch.sqrt(model.Ka[0])

# and pass N_BATCH, Ja0 and Jee to the model
model.N_BATCH = N_BATCH
# copy Ja0
model.Ja0 = Ja0 
# in the model with stp, JEE is J_STP
model.J_STP = JEE # * model.Jab[0, 0]
```

Let's run the simulations

``` ipython
start = perf_counter()
rates = model().cpu().detach().numpy()
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
print('rates', rates.shape)
```

Let's compute the fourier moments of the population activity and reshape
them

``` ipython
idx = get_idx(model)
rates_ordered = rates[..., idx]

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
m0 = m0.reshape(N_JE0, N_JEE, -1)
m1 = m1.reshape(N_JE0, N_JEE, -1)  
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].imshow(m0[..., -5:].mean(-1),
             cmap='jet', origin='lower', vmin=0, vmax=20, aspect='auto',
             extent=[JEE_list[0], JEE_list[-1], JE0_list[0], JE0_list[-1]])

ax[0].set_xlabel('$J_{EE}$')
ax[0].set_ylabel('$J_{E0}$')

ax[1].imshow((m1[...,-5:].mean(-1) - m1[..., :model.N_STIM_ON[0]].mean(-1))
             / m0[...,-5:].mean(-1),
             cmap='jet', origin='lower', vmin=0, vmax=1, aspect='auto',
             extent=[JEE_list[0], JEE_list[-1], JE0_list[0], JE0_list[-1]])

ax[1].set_xlabel('$J_{EE}$')
ax[1].set_ylabel('$J_{E0}$')

plt.show()
```

``` ipython
idx = 6
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(m1[idx].T, alpha=.3)
ax[0].set_ylabel('$\mathcal{F}_1$')
ax[0].set_xlabel('step')
ax[0].set_title('Varying $J_{EE}$')

ax[1].plot(m1[:, idx].T)
ax[1].set_ylabel('$\mathcal{F}_1$')
ax[1].set_xlabel('step')
ax[1].set_title('Varying $J_{E0}$')

plt.show()
```

The parameters corresponding to (row 3, col -1) work!

We can get their values from their matrix form

``` ipython
JEE = JEE.reshape((N_JE0, N_JEE))
JE0 = JE0.reshape((N_JE0, N_JEE))  

print('JE0', JE0[3, -1].item())
print('JEE', JEE[3, -1].item())
```

or directly from the original lists

``` ipython
print('JE0', JE0_list[-1])
print('JEE', JEE_list[-1])
```

### Test

Let's test them.

``` ipython
idx = [3, 10]

model = Network(conf_name, REPO_ROOT, TASK='dual_rand',
                VERBOSE=0, DEVICE='cuda', seed=0, N_BATCH=1, LIVE_FF_UPDATE=1)

# model.Ja0[:, 0] = JE0[idx[0], idx[1]] * model.M0 * torch.sqrt(model.Ka[0])
# model.J_STP = JEE[idx[0], idx[1]]

print(JE0[idx[0], idx[1]].item(), JEE[idx[0], idx[1]].item())
```

``` ipython
rates_test = model().cpu().numpy()
```

``` ipython
idx = get_idx(model)
rates_ordered = rates_test[..., idx]

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
m0, m1, phi = decode_bump(rates_test, axis=-1)
print('m0', m0.shape)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

r_max = 10

ax[0].imshow(rates_ordered[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max, origin='lower')
ax[0].set_ylabel('Neuron #')
ax[0].set_xlabel('Step')

ax[1].plot(m1.T)
ax[1].set_ylabel('$\mathcal{F}_1$')
ax[1].set_xlabel('Step')

plt.show()
```

``` ipython

```